In [1]:
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

from selenium.common.exceptions import TimeoutException

import math

In [2]:
options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=options)
#driver.get("https://www.car.info/en-se/mercedes-benz")
driver.get('https://www.car.info/en-se/ford')

In [96]:
scroll_origin = ScrollOrigin.from_viewport(500, 500)

soup = BeautifulSoup(driver.page_source, "html.parser")

currentSeries = int(driver.find_element(By.XPATH, '//*[@id="hierarchy_list_current"]/td/span/span').text)
historicSeries = int(driver.find_element(By.XPATH, '//*[@id="hierarchy_list_historic"]/td/span/span').text)

for _ in range(100):
        ActionChains(driver)\
            .scroll_from_origin(scroll_origin, 0, 100)\
            .perform()
        time.sleep(0.1)
        try:
            driver.find_element(By.XPATH, f'//*[@id="content"]/main/div[5]/table/tbody/tr[{currentSeries+2}]/td/div/div/div[2]')
            break
        except:
            pass
driver.find_element(By.XPATH, f'//*[@id="content"]/main/div[5]/table/tbody/tr[{currentSeries+2}]/td/div/div/div[2]').click()

for _ in range(100):
        ActionChains(driver)\
            .scroll_from_origin(scroll_origin, 0, 100)\
            .perform()
        time.sleep(0.1)
        try:
            driver.find_element(By.XPATH, f'//*[@id="content"]/main/div[5]/table/tbody/tr[{currentSeries + historicSeries + 4}]/td/div/div/div[2]')
            break
        except:
            pass

driver.find_element(By.XPATH, f'//*[@id="content"]/main/div[5]/table/tbody/tr[{currentSeries + historicSeries + 4}]/td/div/div/div[2]').click()

# for _ in range(100):
#         ActionChains(driver)\
#             .scroll_from_origin(scroll_origin, 0, 100)\
#             .perform()
#         time.sleep(0.1)
#         try:
#             driver.find_element(By.XPATH, '//*[@id="content"]/main/div[6]/div[1]/h2')
#             break
#         except:
#             pass



In [97]:
soup = BeautifulSoup(driver.page_source, "html.parser")
current_series = soup.find_all(
            "table",
            class_="table ci-table table-md table-striped mb-2",
        )
print(len(current_series))
#print(current_series[0])


1


In [98]:
test = str(current_series[0]).split('href="')
links = set()
count = 0
for i in test:
    if "https" in i:
        fullLink = i.split('"')
        partial = fullLink[0].split("en-se/")
        # print(partial)
        split = partial[1].split("/")
        # print(partial[0] + split[0] + "/" + split[1])
        fullLink = partial[0] + split[0] + "/" + split[1]
        #temp = temp[0].split("en-se/")
        links.add(fullLink)
        count+=1
            
print(count)
print(len(links))
links = list(links)
links.sort()
for i in range(10):
    print(links[i])

1002
180
https://www.car.info/ford/300
https://www.car.info/ford/7w
https://www.car.info/ford/7y
https://www.car.info/ford/aerostar
https://www.car.info/ford/anglia
https://www.car.info/ford/aspire
https://www.car.info/ford/b-max
https://www.car.info/ford/bronco
https://www.car.info/ford/bronco-ii
https://www.car.info/ford/bronco-sport


In [59]:
print(links[0])
driver.get(links[0])
soup = BeautifulSoup(driver.page_source, "html.parser")
name = soup.find_all(
            "h1",
            class_="page_title",
        )
name = str(name).split('\n')
name = name[1].split('</h1>')
name = name[0].strip()
name = name.lower().split("mercedes-benz")[1].strip()
print(name)

#print(driver.find_element(By.XPATH, '//*[@id="content"]/main/div[1]/h1'))

https://www.car.info/mercedes-benz/12-55-ps
12/55 ps


In [139]:
driver.get('https://www.car.info/en-se/ford/f-series')
element = driver.find_element(By.XPATH, f"//*[text()='Show all models']").click()

In [9]:
#driver.get('https://www.car.info/en-se/ford/f-series')
#link = 'https://www.car.info/en-se/mercedes-benz/630'
link = 'https://www.car.info/en-se/ford/super-duty'
#driver.get('https://www.car.info/mercedes-benz/12-55-ps')

soup = BeautifulSoup(driver.page_source, "html.parser")

print(f'Fetching link: {link}')
driver.get(link)
soup = BeautifulSoup(driver.page_source, "html.parser")

try:
    driver.find_element(By.XPATH, f"//*[text()='Show all models']").click()
    print("case 1")
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    models = soup.find_all(
            "tr",
            class_="list_item show_condensed show_expanded_list",
        )
    models2 = soup.find_all(
            "tr",
            class_="list_item show_not_condensed show_expanded_list",
        )
    models.extend(models2)
except:
    print("case 2")
    models = soup.find_all(
            "tr",
            class_="list_item show_condensed",
        )
#print(models)
modelLinks = {}
for i in models:
    stringModel = str(i)
    if 'data-type="models"' in stringModel:
        m = stringModel.split('aria-label="')[1].split('<')[0].strip().split('"')[0]
        temp = stringModel.split('href="')
        temp = temp[1].split('">')
        modelLinks[temp[0]] = m #add model link to hashmap
if len(modelLinks) == 0:
    m = driver.find_element(By.XPATH, '//*[@id="content"]/main/div[2]/div[1]/div[1]/div[2]/div[2]').text
    temp = stringModel.split('href="')
    temp = temp[1].split('">')
    modelLinks[link] = m #add model link to hashmap
    
print(modelLinks)
print(f'Total of {len(modelLinks)} models detected')

Fetching link: https://www.car.info/en-se/ford/super-duty
case 2
{'https://www.car.info/en-se/ford/super-duty/f-250-crew-cab-17334651': 'F-250 Crew Cab', 'https://www.car.info/en-se/ford/super-duty/f-250-regular-cab-17334647': 'F-250 Regular Cab', 'https://www.car.info/en-se/ford/super-duty/f-250-supercab-17334649': 'F-250 SuperCab', 'https://www.car.info/en-se/ford/super-duty/f-350-crew-cab-17334657': 'F-350 Crew Cab', 'https://www.car.info/en-se/ford/super-duty/f-350-regular-cab-17334653': 'F-350 Regular Cab', 'https://www.car.info/en-se/ford/super-duty/f-350-supercab-17334655': 'F-350 SuperCab', 'https://www.car.info/en-se/ford/super-duty/f-450-crew-cab-17334663': 'F-450 Crew Cab', 'https://www.car.info/en-se/ford/super-duty/f-450-regular-cab-17334659': 'F-450 Regular Cab', 'https://www.car.info/en-se/ford/super-duty/f-550-crew-cab-17334669': 'F-550 Crew Cab', 'https://www.car.info/en-se/ford/super-duty/f-550-regular-cab-17334665': 'F-550 Regular Cab', 'https://www.car.info/en-se/fo

In [13]:
# driver.get(links[0])
#driver.get('https://www.car.info/en-se/ford/kuga')
driver.get('https://www.car.info/en-se/citroen/jumpy/jumpy-van-7220178')
soup = BeautifulSoup(driver.page_source, "html.parser")
soup = BeautifulSoup(driver.page_source, "html.parser")
try:
    driver.find_element(By.XPATH, f"//*[text()='Show all generations']").click()
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    generations = soup.find_all(
            "tr",
            class_="list_item show_condensed show_expanded_list",
        )
    generations2 = soup.find_all(
            "tr",
            class_="list_item show_not_condensed show_expanded_list",
        )
    generations.extend(generations2)
except:
    generations = soup.find_all(
            "tr",
            class_="list_item show_condensed",
            )
print(f'{len(generations)} generations detected')
genNumerated = 0
generationLabels = {}

for generation in generations:
    type = str(generation)
    if type.split('data-type="')[1].split('"')[0] == "generations": #identify list of generations
        gen = type.split('aria-label="')[1].split('<')[0].strip().split('"')[0]
        generationLabels[gen] = []
        # print(gen)

        tempFuelTypes = ""
        fuel = type.split('make_tooltip" data-tooltip="')
        for i in range(1, len(fuel)):
            tempFuelTypes += fuel[i].split('"')[0].lower()
        print(tempFuelTypes)
        if 'hybrid' in tempFuelTypes:
            generationLabels[gen].append('Hybrid')
        elif 'electric' in tempFuelTypes and 'petrol' in tempFuelTypes:
            generationLabels[gen].append('Electric & Gas')
        elif 'electric' in tempFuelTypes and 'petrol' not in tempFuelTypes:
            generationLabels[gen].append('Electric')
            print('trigger')
        elif 'electric' not in tempFuelTypes and 'petrol' in tempFuelTypes:
            generationLabels[gen].append('Gas')
            print('gas')
        else:
            generationLabels[gen].append('Unknown')

        date = type.split('cell">')[1].split('<')[0].strip().split("-")
        try:
            if int(date[0]) < 1980:
                generationLabels[gen].append("pre-1980")
            else:
                generationLabels[gen].append(date[0])
        except:
            generationLabels[gen].append("ERROR")
        
        try:
            if date[1] == '':
                generationLabels[gen].append("2024")
            elif int(date[1]) < 1980:
                generationLabels[gen].append("pre-1980")
            else:
                generationLabels[gen].append(date[1])
        except:
            generationLabels[gen].append("ERROR")
generationLabels = dict(sorted(generationLabels.items()))
print("-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=- pregen labels")
print(generationLabels)

for g in generationLabels:
    if "Concept" in g:
        generationLabels[g].append("Concept")
    elif "Facelift" not in g:
        genNumerated += 1
        genNumerated = math.floor(genNumerated)
        generationLabels[g].append(math.floor(genNumerated))
    else:
        genNumerated += 0.1
        generationLabels[g].append(f'{genNumerated:.1f}') # will face error if there are more than 10 facelifts for some reason

27 generations detected
petroldiesel
gas
petroldiesel
gas
petroldiesel
gas
petroldiesel
gas
dieselelectricfuel cell
trigger
dieselelectric
trigger
dieselelectric
trigger
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=- pregen labels
{'Jumpy Van 1st Generation': ['Gas', '1995', '2003'], 'Jumpy Van 1st Generation Facelift': ['Gas', '2004', '2007'], 'Jumpy Van 2nd Generation': ['Gas', '2007', '2011'], 'Jumpy Van 2nd Generation Facelift': ['Gas', '2012', '2016'], 'Jumpy Van 3rd Generation': ['Electric', 'ERROR', 'ERROR'], 'Jumpy Van 3rd Generation 1st Facelift': ['Electric', '2019', '2024'], 'Jumpy Van 3rd Generation 2nd Facelift': ['Electric', '2024', '2024']}


In [30]:
x = driver.find_element(By.XPATH, '//*[@id="content"]/main/div[2]/div[1]/div[1]/div[2]/div[2]').text
f = 'F-250 Crew Cab'
x = "Crew Cab"
print(str(x))
if str(x) not in f:
    print("test")
    f = x + " " + f
print(f)

Crew Cab
F-250 Crew Cab


In [71]:
driver.get('https://www.car.info/en-se/ford/f-series')
if len(generations) > 12:
    for _ in range(100):
            ActionChains(driver)\
                .scroll_from_origin(scroll_origin, 0, 100)\
                .perform()
            time.sleep(0.1)
            try:
                driver.find_element(By.XPATH, '//*[@id="hierarchy_table"]/table/tbody/tr[26]/td/div/div/div[2]')
                break
            except:
                pass
    driver.find_element(By.XPATH, '//*[@id="hierarchy_table"]/table/tbody/tr[26]/td/div/div/div[2]').click()


In [144]:
test = temp[4]
test = test.split('make_tooltip" data-tooltip="')
a = ""
for i in range(1, len(test)):
    a += test[i].split('"')[0].lower()
print(a)
if 'hybrid' in a:
    print(True)


petrol


In [182]:

for i in generationLabels:
    print(f'Generation: {generationLabels[i][3]}; Fuel Type: {generationLabels[i][0]}; {generationLabels[i][1]} - {generationLabels[i][2]}')

Generation: 1; Fuel Type: Gas; 1948 - 1950
Generation: 1.1; Fuel Type: Gas; 1951 - 2024
Generation: 1.2; Fuel Type: Gas; 1952 - 2024
Generation: 2; Fuel Type: Gas; 1953 - 2024
Generation: 2.1; Fuel Type: Gas; 1954 - 2024
Generation: 2.2; Fuel Type: Gas; 1955 - 2024
Generation: 2.3; Fuel Type: Gas; 1956 - 2024
Generation: 3; Fuel Type: Gas; 1957 - 2024
Generation: 3.1; Fuel Type: Gas; 1958 - 2024
Generation: 3.2; Fuel Type: Gas; 1959 - 2024
Generation: 3.3; Fuel Type: Gas; 1960 - 2024
Generation: 4; Fuel Type: Gas; 1961 - 2024
Generation: 4.1; Fuel Type: Gas; 1962 - 2024
Generation: 4.2; Fuel Type: Gas; 1963 - 2024
Generation: 4.3; Fuel Type: Gas; 1964 - 2024
Generation: 4.4; Fuel Type: Gas; 1965 - 2024
Generation: 4.5; Fuel Type: Gas; 1966 - 2024
Generation: 5; Fuel Type: Gas; 1967 - 2024
Generation: 5.1; Fuel Type: Gas; 1968 - 2024
Generation: 5.2; Fuel Type: Gas; 1969 - 2024
Generation: 5.3; Fuel Type: Gas; 1970 - 2024
Generation: 5.4; Fuel Type: Gas; 1971 - 2024
Generation: 5.5; Fue

In [4]:
job_list = []

def search_job_page(keywords):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    scroll_origin = ScrollOrigin.from_viewport(200, 500)
    
    for _ in range(3):
        ActionChains(driver)\
            .scroll_from_origin(scroll_origin, 0, 2000)\
            .perform()
        
        soup = BeautifulSoup(driver.page_source, "html.parser")
        job_listings = soup.find_all(
                    "div",
                    class_="job-card-list__entity-lockup artdeco-entity-lockup artdeco-entity-lockup--size-4 ember-view",
                )
        try:
            driver.find_element(By.XPATH, f"//*[@id='{job_listings[-1].get('id')}']").click()
        except:
            pass
        time.sleep(0.2)
        
    job_listings = soup.find_all(
            "div",
            class_="job-card-list__entity-lockup artdeco-entity-lockup artdeco-entity-lockup--size-4 ember-view",
        )
    
    print(f'Numer of jobs detected on page: {len(job_listings)}')

    for job in job_listings:
        driver.find_element(By.XPATH, f"//*[@id='{job.get('id')}']").click()
        time.sleep(0.1)
        load_job_data(keywords)

In [4]:
def load_job_data(keywords):
    soup = BeautifulSoup(driver.page_source, "html.parser")

    #needs to be adjusted based on screen size; this is calibrated for vertical monitor fullscreen
    scroll_origin = ScrollOrigin.from_viewport(500, 500)
    
    ActionChains(driver)\
        .scroll_from_origin(scroll_origin, 0, -200)\
        .perform()
    time.sleep(0.1)
    for _ in range(5):
        ActionChains(driver)\
            .scroll_from_origin(scroll_origin, 0, 1000)\
            .perform()
        time.sleep(0.1)
    
    timeout = 2
    try:
        element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div[2]/div[2]/div/div[2]/div/div[1]/div/section/section/div[1]/div[2]'))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
    
    description = driver.find_element(By.XPATH, '//*[@id="job-details"]')
    time.sleep(0.5)
    for word in keywords:
        if word.lower() in description.text.lower():
            try:
                #job_company = soup.find('div', {'class': 'artdeco-entity-lockup__title ember-view t-20'}).text.strip()
                job_company = soup.find('a', {'class': 'ember-view link-without-visited-state inline-block t-black'}).text.strip()
            except:
                job_company = 'N/A'
            job_name = soup.find('span', {'class': 'job-details-jobs-unified-top-card__job-title-link'}).text.strip()
            #job_time = soup.find('span', {'class': 'tvm__text tvm__text--positive'}).text.strip()
            job_link = driver.current_url

            job_list.append({"Company" : job_company, "Job Title" : job_name, "Link" : job_link})
            break
        else:
            pass

In [8]:
def scrape_jobs(title, location, keywords):
    page = 0
    while True:
        driver.get(f"https://www.linkedin.com/jobs/search/?keywords={title}&location={location}&start={page*25}")
        
        try:
            driver.find_element(By.XPATH, "/html/body/div[4]/div[3]/div[4]/div/div[1]/div/div[1]")
            print("Search Complete.")
            break
        except:
            time.sleep(1)
            search_job_page(keywords)
            page += 1

In [1]:
#scrape_jobs("machine learning", "Remote", ["high school", "highschool"])
#print(job_list)

In [4]:
from pyexcel.cookbook import merge_all_to_a_book
# import pyexcel.ext.xlsx # no longer required if you use pyexcel >= 0.2.2 
import glob


merge_all_to_a_book(glob.glob("your_csv_directory/*.csv"), "output.xlsx")

SupportingPluginAvailableButNotInstalled: Please install one of these plugins for read data in 'xlsx': pyexcel-xlsx,pyexcel-xlsxw

In [3]:
%pip install pyexcel-xlsx

  Using cached pyexcel_xlsx-0.6.0-py2.py3-none-any.whl.metadata (16 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached pyexcel_xlsx-0.6.0-py2.py3-none-any.whl (9.6 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: c:\Users\jonso\.pyenv\pyenv-win\versions\3.9.13\python.exe -m pip install --upgrade pip


## Search Companies for Keywords